In [50]:
import numpy as np

In [63]:
# 定义全局常量
G = 9.81

# 定义Junction节点类
class Junction:
    def __init__(self):
        self.ki = 0 
        self.ko = 0
        self.flag = 0
        self.in_ = np.zeros(10,dtype=int)
        self.out = np.zeros(10,dtype=int)
        self.z = 0.0
        self.dz = 0.0
        self.o = 0.0
        self.a = 0.0

# 定义Channel河段类        
class Channel:
    def __init__(self):
        self.name = ''
        self.no = 0
        self.st = 0 
        self.en = 0
        self.m = 0
        self.slope = 0.0
        self.la = 0.0
        self.ds = 0.0
        self.s = np.zeros(1000)
        self.zb = np.zeros(1000)
        # ...省略其它成员变量初始化...
        
# 定义全局变量        
JuncNum = 0 
ChanNum = 0
ChanM = 100
SecN = 1000
JuncM = 100
JuncN = 10 

Chan = [Channel() for i in range(ChanM)] 
Junc = [Junction() for i in range(JuncM)]

In [51]:
# 定义变量
m = 1000 
theta = 0.75
alp = 1.0

In [52]:
# 定义文件夹路径
folder = '/home/louyaxin/1D-Hydrodynamic-Model/input/'
folder1 = 'result/'

In [53]:
# 打开拓扑文件
def read_file(f):
    with open(folder + 'TOPO.txt') as f:
        JuncNum, JuncNum1, JuncNum2 = [int(x) for x in f.readline().split()] 
        ChanNum, chanNum1, chanNum2 = [int(x) for x in f.readline().split()]
        UbouNum, AbouNum = [int(x) for x in f.readline().split()]

In [54]:
print(f'ChanNum={ChanNum}, chanNum1={chanNum1}, chanNum2={chanNum2}') 
print(f'JuncNum={JuncNum}, JuncNum1={JuncNum1}, JuncNum2={JuncNum2}')
print('river star end')

ChanNum=3, chanNum1=0, chanNum2=1
JuncNum=4, JuncNum1=1, JuncNum2=2
river star end


In [55]:
read_file(f)

In [56]:
def read_file(f):
    chan = []
    for i in range(ChanNum):
        no, st, en, name = f.readline().split()
        chan.append({
       'no': int(no),
       'st': int(st),
       'en': int(en),
       'name': name
    })
    print(chan[i]['no'], chan[i]['st'], chan[i]['en'], chan[i]['name'])
print('Handle the relationship between river section number and node number:')   

Handle the relationship between river section number and node number:


In [58]:
#打开时间步长文件
with open(folder + 'TIME.txt') as f:
    dt = float(f.readline())  
    rdt = float(f.readline())

In [59]:
# 根据Junc中的标记分类节点
for i in range(JuncNum1+1, JuncNum2):
    Junc[i].flag = 1
    print(f'Nodes {i} is downstream boundary point')

In [64]:
for i in range(JuncNum1+1, JuncNum2+1):
    Junc[i]['flag'] = 1
    print(f'Nodes {i} is downstream boundary point')

TypeError: 'Junction' object does not support item assignment

In [70]:
import numpy as np

# 初始化参数
JuncNum = 0 
ChanNum = 0
JuncNum1 = 0
JuncNum2 = 0
UbouNum = 0
AbouNum = 0

# 读入河段和节点数据
with open('/home/louyaxin/1D-Hydrodynamic-Model/input/TOPO.txt') as f:
    JuncNum, JuncNum1, JuncNum2 = [int(x) for x in f.readline().split()]  
    ChanNum, chanNum1, chanNum2 = [int(x) for x in f.readline().split()]
    UbouNum, AbouNum = [int(x) for x in f.readline().split()]

    
print(f'ChanNum={ChanNum}, chanNum1={chanNum1}, chanNum2={chanNum2}')
print(f'JuncNum={JuncNum}, JuncNum1={JuncNum1}, JuncNum2={JuncNum2}')

def read_file(f):
    chan = [] 
    for i in range(ChanNum):
        no, st, en, name = f.readline().split()
        chan.append({'no': int(no), 'st': int(st), 'en': int(en), 'name': name})

# 分类节点
Junc = np.zeros(JuncNum, dtype=[('no', 'int'), ('flag', 'int')])
for i in range(JuncNum1+1, JuncNum2):
    Junc[i]['flag'] = 1
    print(f'Nodes {i} is downstream boundary point')
    
for i in range(JuncNum2+1, JuncNum):
    Junc[i]['flag'] = -1
    print(f'Nodes {i} is upstream boundary point')

# 保存边界条件    
Bou = []
for i in range(JuncNum):
    if Junc[i]['flag'] != 0:
        b = {}
        b['No'] = i
        b['flag'] = Junc[i]['flag']
        if Junc[i]['flag'] == 1:
            b['nc'] = Junc[i]['out']  
        elif Junc[i]['flag'] == -1:
            b['nc'] = Junc[i]['in']
        Bou.append(b)

BouNum = len(Bou)
print(f'Number of boundary points: {BouNum}') 

for i in range(BouNum):
    print(i, Bou[i]['No'], Bou[i]['flag'], Bou[i]['nc'])

print('Done')

ChanNum=3, chanNum1=0, chanNum2=1
JuncNum=4, JuncNum1=1, JuncNum2=2
Nodes 3 is upstream boundary point


ValueError: no field of name in